In [1943]:
dramas_cleaned.to_parquet("dramas_preprocessed.parquet", index=False)

In [1944]:
dramas_cleaned = pd.read_parquet("dramas_preprocessed.parquet")

In [1945]:
(dramas_cleaned.isna().mean() * 100).sort_values(ascending=False)


imdb_rating_count                   0.0
imdb_rating                         0.0
in_production                       0.0
last_episode_to_air_vote_average    0.0
series_id                           0.0
original_name                       0.0
popularity                          0.0
executive_producer_name             0.0
executive_producer_ids              0.0
executive_producer_gender           0.0
executive_producer_profile_path     0.0
writers_name                        0.0
writer_roles                        0.0
writer_ids                          0.0
writer_gender                       0.0
writer_profile_path                 0.0
top_cast_order                      0.0
top_cast                            0.0
character                           0.0
top_cast_ids                        0.0
top_cast_gender                     0.0
id                                  0.0
episode_run_time                    0.0
number_of_seasons                   0.0
genre_ids                           0.0


In [1946]:
dramas_cleaned["episode_run_time"].head(30)

0                     42
1                     27
2                     30
3                     44
4                     55
5                 41, 43
6                     44
7                 60, 43
8                     45
9                     24
10                45, 50
11                    45
12                60, 46
13                    41
14    44, 46, 47, 50, 60
15                    60
16                    55
17                    44
18                    52
19                60, 55
20                    28
21                42, 60
22                44, 45
23                    43
24                     0
25                    45
26                    40
27                    41
28                    54
29                    34
Name: episode_run_time, dtype: object

In [1947]:
# 문자열을 분리해서 episode_run_time 평균 구하기
dramas_cleaned['episode_run_time_average'] = dramas_cleaned['episode_run_time'].apply(
    lambda x: sum(map(int, x.split(','))) / len(x.split(','))
)

print(dramas_cleaned['episode_run_time_average'].head())

0    42.0
1    27.0
2    30.0
3    44.0
4    55.0
Name: episode_run_time_average, dtype: float64


In [1949]:
dramas_cleaned.columns

Index(['imdb_rating_count', 'imdb_rating', 'keyword', 'imdb_id',
       'production_companies', 'poster_path', 'type_detail', 'seasons',
       'providers_flatrate', 'overview', 'genre_ids', 'genres',
       'origin_country', 'last_air_date', 'first_air_date',
       'number_of_episodes', 'title', 'status', 'vote_average', 'vote_count',
       'original_language', 'number_of_seasons', 'episode_run_time', 'id',
       'in_production', 'last_episode_to_air_vote_average', 'series_id',
       'original_name', 'popularity', 'executive_producer_name',
       'executive_producer_ids', 'executive_producer_gender',
       'executive_producer_profile_path', 'writers_name', 'writer_roles',
       'writer_ids', 'writer_gender', 'writer_profile_path', 'top_cast_order',
       'top_cast', 'character', 'top_cast_ids', 'top_cast_gender',
       'top_cast_profile_path', 'episode_run_time_average'],
      dtype='object')

In [1950]:
dramas_cleaned[['id', 'series_id', 'imdb_id', 'title', 'original_language', 'overview', 'first_air_date', 'episode_run_time_average', 'genres', 'keyword', 'poster_path', 'vote_average', 'vote_count', 'imdb_rating', 'imdb_rating_count']].rename(columns={'vote_average':'tmdb_rating', 'vote_count':'tmdb_num_votes'}).to_parquet('files/dramas_filtered.parquet')

In [1951]:
# 리스트 평탄화
def flatten_and_to_set(nested_lists):
    return list(set(list(itertools.chain.from_iterable(nested_lists))))

# 데이터 파싱
def optimized_provider_parse(data, column):

    dict_series = data[column].apply(ast.literal_eval)

    nested_lists_series = dict_series.apply(lambda x: list(x.values()))

    result = nested_lists_series.apply(flatten_and_to_set)

    return result

def table_normalization(data, column_list):
    nom_data = {}
    for column in column_list:
        data_parsed = data[['imdb_id', column]]
        nom_data[column] = data_parsed.explode(column).reset_index(drop=True)
        nom_data[column].to_parquet("files/" + column + ".parquet")

    print("행 정규화 완료")

def parsing_columns(data, columns):
    for column in columns:
        data[column] = data[column].str.split(', ')
    return data

In [1952]:
dramas_cleaned_1 = dramas_cleaned.copy()
parsing_col = ['origin_country', 'genre_ids', 'keyword']
parsing_columns(dramas_cleaned_1, parsing_col)

,imdb_rating_count,imdb_rating,keyword,imdb_id,production_companies,poster_path,type_detail,seasons,providers_flatrate,overview,...,writer_ids,writer_gender,writer_profile_path,top_cast_order,top_cast,character,top_cast_ids,top_cast_gender,top_cast_profile_path,episode_run_time_average
0,32826.0,7.2,"[medium, psychic power, paranormal, criminal consultant, phoenix, arizona]",tt0412175,"Pointe Studios, Picturemaker Productions, Grammnet Productions, Paramount Television, CBS Studios",/6tOxspW2MbbAQ0s3jtfs8QATcx.jpg,Scripted,S0: Specials (30 eps); S1: Season 1 (16 eps); S2: Season 2 (22 eps); S3: Season 3 (21 eps); S4: Season 4 (16 eps); S5: Season 5 (18 eps); S6: Season 6 (22 eps); S7: Season 7 (13 eps),"{""AR"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel "", ""Paramount+ Amazon Channel""], ""AU"": [""Paramount Plus"", ""Paramount+ Amazon Channel"", ""Paramount Plus Apple TV Channel "", ""Paramount Plus Premium"", ""Paramount Plus Basic with Ads""], ""BO"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""BR"": [""Paramount Plus"", ""Paramount+ Amazon Channel"", ""Paramount Plus Apple TV Channel "", ""Paramount Plus Premium""], ""CA"": [""Paramount Plus"", ""Paramount+ Amazon Channel"", ""Paramount Plus Apple TV Channel "", ""Paramount Plus Premium"", ""Paramount Plus Basic with Ads""], ""CL"": [""Paramount Plus"", ""MovistarTV"", ""Paramount Plus Apple TV Channel "", ""Paramount+ Amazon Channel""], ""CO"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel "", ""Paramount+ Amazon Channel""], ""CR"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""CZ"": [""Oneplay""], ""DO"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""EC"": [""Paramount Plus"", ""MovistarTV"", ""Paramount Plus Apple TV Channel ""], ""FR"": [""M6+"", ""Molotov TV""], ""GT"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""HN"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""JP"": [""Hulu"", ""U-NEXT""], ""MX"": [""Paramount Plus"", ""Paramount+ Amazon Channel"", ""Paramount Plus Apple TV Channel "", ""Paramount Plus Premium""], ""NI"": [""Paramount Plus Apple TV Channel "", ""Paramount Plus""], ""PA"": [""Paramount Plus Apple TV Channel "", ""Paramount Plus""], ""PE"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""PY"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""SV"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""], ""US"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel "", ""Paramount+ Amazon Channel"", ""Paramount+ Roku Premium Channel""], ""UY"": [""Paramount Plus""], ""VE"": [""Paramount Plus"", ""Paramount Plus Apple TV Channel ""]}",Allison Dubois works in the District Attorney’s office using her natural intuition about people and her ability to communicate with the dead to help to solve crimes. Her dreams often give her clues to the whereabouts of missing people.,...,1318326,2,/4hEMgTX1BBXXNPGNFga8xY3e7MS.jpg,0; 1; 2; 3; 4,Patricia Arquette; Jake Weber; Miguel Sandoval; David Cubitt; Sofia Vassilieva,Allison Dubois; Joe Dubois; Manuel Devalos; Lee Scanlon; Ariel Dubois,4687; 4177; 30488; 155282; 95039,1; 2; 2; 2; 1,/jeThSouMatiuRiLkjDvSBLHpmq0.jpg; /z1RTh4qvyMFha8nKd5Uw1dKwkZz.jpg; /v5tHxTkgOqcdWgjoEhUj0a50NIK.jpg; /2zR5HQnOFLB6tvgmKofuk5J0E4D.jpg; /ryZqVhzjKfUBYuPMKXvZ1KF4tmU.jpg,42.0
1,121234.0,7.9,"[drug dealer, outlaw, dysfunctional family, dark comedy, marijuana, single mother, drugs, family conflict, fast food, simulated sex, daily drama]",tt0439100,"Tilted Productions, Lionsgate Television",/gKUsBTa5b0GY7U4iWBseeBalfjO.jpg,Scripted,S0: Specials (1 eps); S1: Season 1 (10 eps); S2: Season 2 (12 eps); S3: Season 3 (15 eps); S4: Season 4 (13 eps); S5: Season 5 (13 eps); S6: Season 6 (13 eps); S7: Season 7 (13 eps); S8: Season 8 (13 eps),"{""AD"": [""Netflix""], ""AE"": [""Netflix""], ""AG"": [""Netflix""], ""AO"": [""Netflix""], ""AR"": [""Netflix""], ""AT"": [""Lionsgate+ Amazon Channels"", ""Netflix""], ""AU

In [1953]:
import json
import ast
import itertools

def safe_parse_dict(x):
    if pd.isna(x):
        return {}
    
    # 이미 dict라면 그대로 반환
    if isinstance(x, dict):
        return x
    
    # 문자열 양끝 공백 제거
    s = str(x).strip()
    
    # dict 형태가 아니면 빈 dict로 처리
    if not (s.startswith("{") and s.endswith("}")):
        return {}
    
    # literal_eval 시도
    try:
        return ast.literal_eval(s)
    except Exception:
        # literal_eval 실패 → JSON 스타일로 변환 시도
        try:
            return json.loads(s.replace("'", '"'))
        except Exception:
            return {}  # 그래도 실패하면 빈 dict 처리


def optimized_provider_parse1(data, column):
    dict_series = data[column].apply(safe_parse_dict)

    nested_lists_series = dict_series.apply(lambda x: list(x.values()))

    result = nested_lists_series.apply(flatten_and_to_set)

    return result


In [1954]:

dramas_cleaned_1['providers_flatrate'] = optimized_provider_parse1(dramas_cleaned_1, "providers_flatrate")

table_normalization(dramas_cleaned_1, parsing_col + ['providers_flatrate'])


행 정규화 완료


In [1955]:
dramas_cleaned_1['providers_flatrate'].head(10)

0    [Paramount+ Amazon Channel, MovistarTV, Paramount Plus Basic with Ads, Paramount+ Roku Premium Channel, Paramount Plus Apple TV Channel , M6+, Paramount Plus Premium, Paramount Plus, Hulu, U-NEXT, Molotov TV, Oneplay]
1                                                                        [Amediateka, Lionsgate+ Amazon Channels, Stan, Netflix Standard with Ads, Starz Amazon Channel, MovieSphere+ Amazon Channel, Netflix, Lionsgate Play]
2                                                                                                                                                                      [Netflix, Netflix Standard with Ads, TVNZ+, Amediateka]
3                                                                                                                                                                                                          [Hulu, Disney Plus]
4                                                                                                           

In [1956]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import ast
import itertools

In [1957]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

# TMDB 영화 장르 목록을 가져오는 API 엔드포인트
URL = f"https://api.themoviedb.org/3/genre/tv/list?api_key={API_KEY}&language=en"

try:
    # API 요청 보내기
    response = requests.get(URL)
    response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    # JSON 데이터 파싱
    data = response.json()

    # 장르 목록 추출
    genres = data.get('genres', [])

    if genres:
        genres_df = pd.DataFrame(genres).rename(columns={'id':'genre_id', 'name':'genre'})
    else:
        print("❌ 장르 데이터를 찾을 수 없거나 응답이 비어 있습니다.")

except requests.exceptions.HTTPError as err:
    print(f"❌ HTTP 오류 발생: {err}")
    print("API 키를 확인하거나 요청 URL을 확인해 보세요.")
except requests.exceptions.RequestException as err:
    print(f"❌ 요청 오류 발생: {err}")
except json.JSONDecodeError:
    print("❌ 응답을 JSON으로 디코딩하는 데 실패했습니다.")

genre_ids = pd.read_parquet('files/genre_ids.parquet').rename(columns={'genre_ids':"genre_id"})
genre_ids['genre_id'] = genre_ids['genre_id'].astype('int')
pd.merge(genre_ids, genres_df, on = 'genre_id', how = 'left').to_parquet('files/genres.parquet')

### genres_id, genre가 모두 포함된 genres.parquet 파일이 생성!

In [1978]:
genres = pd.read_parquet('files/genres.parquet')
genres.shape

(7081, 3)

In [1977]:
genres['genre'].head()

0               Crime
1               Drama
2             Mystery
3    Sci-Fi & Fantasy
4              Comedy
Name: genre, dtype: object

In [1959]:
origin_country = pd.read_parquet('files/origin_country.parquet')
origin_country.head()


,imdb_id,origin_country
0,tt0412175,US
1,tt0439100,US
2,tt0772137,US
3,tt0756509,US
4,tt0496275,US


In [1979]:
origin_country.shape

(3472, 2)

In [1960]:
origin_country_cnt = origin_country['origin_country'].value_counts()
origin_country.loc[origin_country['origin_country'].isin(origin_country_cnt[10:].index), 'origin_country'] = 'Other'
origin_country['origin_country'] = origin_country['origin_country'].str.lower()
origin_country = origin_country.drop_duplicates()
origin_country.head(30)


,imdb_id,origin_country
0,tt0412175,us
1,tt0439100,us
2,tt0772137,us
3,tt0756509,us
4,tt0496275,us
5,tt0758737,us
6,tt0435576,us
7,tt0805663,us
8,tt0805666,us
9,tt0452568,gb


In [1961]:
origin_country.to_parquet('files/origin_country_f.parquet')

In [1962]:
dramas_filtered = pd.read_parquet('files/dramas_filtered.parquet')


In [1963]:
dramas_filtered.loc[~dramas_filtered['original_language'].isin(dramas_filtered['original_language'].value_counts()[:10].index), 'original_language'] = 'xx'

### 최종 dramas_filtered_f 파일 생성

In [1964]:
dramas_filtered.to_parquet('files/dramas_filtered_f.parquet')